# pyPAGE: Comprehensive Tutorial

This notebook provides an end-to-end tutorial for **pyPAGE**, a Python implementation of the conditional-information PAGE framework for gene-set enrichment analysis. It covers:

1. Loading gene sets (paired arrays, annotation files, GMT files)
2. Loading expression data (continuous scores, pre-binned labels)
3. Gene ID conversion with `GeneMapper` (offline, cached)
4. Bulk PAGE analysis (MI/CMI, permutation testing, redundancy filtering, visualization)
5. Single-cell PAGE analysis (per-cell scoring, spatial coherence, neighborhood mode)
6. A complete integrated workflow combining GMT loading, gene ID mapping, and PAGE

## Installation

Install pyPAGE from PyPI:

```bash
pip install bio-pypage
```

Or install from source for development:

```bash
git clone https://github.com/goodarzilab/pyPAGE
cd pyPAGE
pip install -e .
```

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pypage import PAGE, ExpressionProfile, GeneSets, GeneMapper, SingleCellPAGE

# For reproducibility
np.random.seed(42)

---
## 1. Loading Gene Sets

`GeneSets` stores pathway/regulon annotations as a binary membership matrix. There are several ways to create one.

### 1a. From Paired Arrays

The simplest approach — provide two arrays of equal length: one with gene names, one with pathway names. Each pair defines a gene-pathway membership.

In [ ]:
genes_arr = np.array(['TP53', 'TP53', 'BRCA1', 'BRCA1', 'MYC', 'MYC', 'EGFR'])
pathways_arr = np.array(['Apoptosis', 'P53_Signaling', 'DNA_Repair', 'Apoptosis',
                         'Cell_Cycle', 'Apoptosis', 'Cell_Cycle'])

gs_arrays = GeneSets(genes=genes_arr, pathways=pathways_arr)
print(gs_arrays)
print("Genes:", gs_arrays.genes)
print("Pathways:", gs_arrays.pathways)
print("Bool array shape:", gs_arrays.bool_array.shape)

### 1b. From Long-Format Files (Paired Arrays)

Many annotation files are in long format — one gene-pathway pair per line. Load these with `pd.read_csv` and pass the two columns as paired arrays. The bundled GO Biological Process data is in this format:

In [ ]:
ann = pd.read_csv(
    "../example_data/GO_BP_2021_index.txt.gz",
    sep="\t", header=None, names=["gene", "pathway"]
)
print("Annotation file format (gene-pathway pairs):")
print(ann.head())

gs_ann = GeneSets(ann["gene"], ann["pathway"])
print(f"\n{gs_ann}")
print(f"Example pathways: {gs_ann.pathways[:3]}")
print(f"Example genes: {gs_ann.genes[:10]}")

For **index-format** annotation files (one pathway per line, followed by all its genes), use `ann_file=`:

```python
# Pathway-first index file: PathwayA\tGene1\tGene2\tGene3
gs = GeneSets(ann_file="pathways_index.txt")

# Gene-first index file: Gene1\tPathwayA\tPathwayB
gs = GeneSets(ann_file="genes_index.txt", first_col_is_genes=True)
```

### 1c. From GMT Files

MSigDB distributes gene sets in GMT format. `GeneSets.from_gmt()` reads `.gmt` or `.gmt.gz` files directly:

In [ ]:
gs_gmt = GeneSets.from_gmt("../example_data/example.gmt")
print(gs_gmt)
print("Pathways:", gs_gmt.pathways)
print("Total genes:", gs_gmt.n_genes)
print("\nPathway sizes:")
for pw in gs_gmt.pathways:
    idx = np.where(gs_gmt.pathways == pw)[0][0]
    size = gs_gmt.bool_array[idx].sum()
    print(f"  {pw}: {size} genes")

GMT files store an optional description field (second column). After loading, descriptions are accessible via the `descriptions` attribute:

In [ ]:
for pathway, desc in gs_gmt.descriptions.items():
    print(f"{pathway}: {desc}")

### 1d. Filtering Pathways by Size

You can filter pathways by minimum/maximum gene count, either at load time or afterwards:

In [ ]:
# Filter at load time
gs_filtered = GeneSets.from_gmt("../example_data/example.gmt", min_size=15, max_size=500)
print(f"After filtering: {gs_filtered.n_pathways} pathways")

# Or filter after loading
gs_gmt2 = GeneSets.from_gmt("../example_data/example.gmt")
print(f"Before filter: {gs_gmt2.n_pathways} pathways")
gs_gmt2.filter_pathways(min_size=18)
print(f"After filter (min_size=18): {gs_gmt2.n_pathways} pathways")

### 1e. Exporting to GMT

Export any `GeneSets` object to GMT format with `to_gmt()`. This enables round-trip loading and saving:

In [ ]:
import tempfile, os

# Round-trip: load -> export -> reload
gs_original = GeneSets.from_gmt("../example_data/example.gmt")

with tempfile.TemporaryDirectory() as tmpdir:
    out_path = os.path.join(tmpdir, "exported.gmt")
    gs_original.to_gmt(out_path)
    
    # Reload and verify
    gs_reloaded = GeneSets.from_gmt(out_path)
    print(f"Original: {gs_original.n_pathways} pathways, {gs_original.n_genes} genes")
    print(f"Reloaded: {gs_reloaded.n_pathways} pathways, {gs_reloaded.n_genes} genes")
    print(f"Pathways match: {set(gs_original.pathways) == set(gs_reloaded.pathways)}")
    print(f"Genes match: {set(gs_original.genes) == set(gs_reloaded.genes)}")

### 1f. Inspecting a GeneSets Object

Key attributes:
- `genes` — sorted array of gene names
- `pathways` — sorted array of pathway names
- `bool_array` — binary matrix of shape `(n_pathways, n_genes)`
- `membership` — per-gene count of pathway memberships (used for CMI bias correction)
- `n_genes`, `n_pathways` — counts

In [ ]:
gs = GeneSets.from_gmt("../example_data/example.gmt")
print(gs)
print(f"Bool array shape: {gs.bool_array.shape}")
print(f"Membership (first 10 genes): {gs.membership[:10]}")
print(f"Genes in multiple pathways: {(gs.membership > 1).sum()}")

---
## 2. Loading Expression Data

`ExpressionProfile` holds gene expression data. It accepts either continuous scores (which are auto-discretized into bins) or pre-binned integer labels.

### 2a. Continuous Scores (Auto-Binned)

When `is_bin=False` (the default), continuous expression values are discretized into `n_bins` equal-frequency bins when `get_gene_subset()` is called:

In [ ]:
# Synthetic continuous expression data
gene_names = np.array([f'Gene{i}' for i in range(200)])
scores = np.random.randn(200)  # continuous differential expression scores

exp_continuous = ExpressionProfile(gene_names, scores, is_bin=False, n_bins=10)
print(exp_continuous)
print(f"Raw expression range: [{scores.min():.2f}, {scores.max():.2f}]")

### 2b. Pre-Binned Labels

When `is_bin=True`, the expression array is treated as integer bin assignments. This is common when working with pre-processed rank data:

In [ ]:
# Load the example pre-binned data
expr_df = pd.read_csv(
    "../example_data/AP2S1.tab.gz",
    sep="\t", header=None, names=["gene", "bin"]
)
print(expr_df.head())

exp_binned = ExpressionProfile(expr_df["gene"], expr_df["bin"], is_bin=True)
print(f"\n{exp_binned}")
print(f"Unique bins: {np.unique(expr_df['bin'])}")

### 2c. Inspecting an ExpressionProfile

Key attributes:
- `genes` — gene names
- `n_genes` — number of genes
- `n_bins` — number of bins
- `raw_expression` — the original expression/bin values

In [ ]:
print(exp_binned)
print(f"Genes (first 5): {exp_binned.genes[:5]}")
print(f"Expression shape: {exp_binned.raw_expression.shape}")

---
## 3. Gene ID Conversion

Gene sets and expression data sometimes use different ID types (Ensembl IDs, gene symbols, Entrez IDs). pyPAGE provides two approaches for conversion.

### 3a. GeneMapper — Offline Cached Mapping (Recommended)

`GeneMapper` downloads a gene ID mapping table from Ensembl BioMart once and caches it locally (~5 MB at `~/.pypage/`). Subsequent calls load from cache with no network needed.

**Supported ID types:** `'ensg'`, `'symbol'`, `'entrez'`

For this tutorial, we'll create a small mock cache file so the notebook runs fully offline:

In [ ]:
import tempfile, os

# Create a mock gene mapping cache (normally GeneMapper downloads this from Ensembl)
mock_cache_dir = tempfile.mkdtemp()
mock_tsv = os.path.join(mock_cache_dir, "gene_map_human.tsv")

mock_data = """ensg\tsymbol\tentrez
ENSG00000141510\tTP53\t7157
ENSG00000012048\tBRCA1\t672
ENSG00000136997\tMYC\t4609
ENSG00000146648\tEGFR\t1956
ENSG00000157764\tBRAF\t673
ENSG00000171862\tPTEN\t5728
ENSG00000133703\tKRAS\t3845
ENSG00000149311\tATM\t472
ENSG00000139618\tBRCA2\t675
ENSG00000116044\tNFE2L2\t4780
ENSG00000164362\tTERT\t7015
ENSG00000118058\tKMT2A\t4297
ENSG00000181555\tSETBP1\t26040
ENSG00000168036\tCTNNB1\t1499
ENSG00000196712\tNF1\t4763
ENSG00000134086\tVHL\t7428
ENSG00000183765\tCASP3\t836
ENSG00000164305\tCASP7\t840
ENSG00000064012\tCASP8\t841
ENSG00000132906\tCASP9\t842
ENSG00000087088\tBAX\t581
ENSG00000030110\tBAK1\t578
ENSG00000171791\tBCL2\t596
ENSG00000171552\tBCL2L1\t598
ENSG00000015475\tBID\t637
ENSG00000172115\tCYCS\t54205
ENSG00000120868\tAPAF1\t317
ENSG00000184047\tDIABLO\t56616
ENSG00000101966\tXIAP\t331
ENSG00000110330\tBIRC2\t329
ENSG00000023445\tBIRC3\t330
ENSG00000168040\tFADD\t8772
ENSG00000026103\tFAS\t355
ENSG00000117560\tFASLG\t356
ENSG00000104689\tTNFRSF10A\t8797
ENSG00000120889\tTNFRSF10B\t8795
ENSG00000073111\tMDM2\t4193
ENSG00000198625\tMDM4\t4194
ENSG00000124762\tCDKN1A\t1026
ENSG00000147889\tCDKN2A\t1029
"""

with open(mock_tsv, 'w') as f:
    f.write(mock_data)

print(f"Mock cache created at: {mock_cache_dir}")

In [ ]:
# Create a GeneMapper pointing at our mock cache
mapper = GeneMapper(species='human', cache_dir=mock_cache_dir)

# Convert Ensembl IDs to gene symbols
ensg_ids = ['ENSG00000141510', 'ENSG00000012048', 'ENSG00000136997', 'ENSG00000000000']
symbols, unmapped = mapper.convert(ensg_ids, from_type='ensg', to_type='symbol')

print("Input Ensembl IDs:", ensg_ids)
print("Converted symbols:", symbols)
print("Unmapped IDs:", unmapped)

In [ ]:
# Convert gene symbols to Entrez IDs
symbols_in = ['TP53', 'BRCA1', 'MYC']
entrez_ids, unmapped = mapper.convert(symbols_in, from_type='symbol', to_type='entrez')

print("Symbols:", symbols_in)
print("Entrez IDs:", entrez_ids)

In [ ]:
# GeneMapper also handles versioned Ensembl IDs (strips the version suffix)
versioned = ['ENSG00000141510.15', 'ENSG00000012048.23']
result, _ = mapper.convert(versioned, from_type='ensg', to_type='symbol')
print("Versioned IDs:", versioned)
print("Resolved:", result)

### 3b. GeneSets.map_genes() — Convert Gene IDs In-Place

`map_genes()` uses a `GeneMapper` to convert all gene IDs in a `GeneSets` object. Unmapped genes are dropped automatically.

In [ ]:
# Create a GeneSets with Ensembl IDs (using genes from our mock cache)
ensg_genes = np.array([
    'ENSG00000183765', 'ENSG00000183765',  # CASP3 -> Apoptosis, P53_Pathway
    'ENSG00000087088', 'ENSG00000087088',  # BAX -> Apoptosis, P53_Pathway
    'ENSG00000141510', 'ENSG00000141510',  # TP53 -> P53_Pathway, DNA_Repair
    'ENSG00000012048',                       # BRCA1 -> DNA_Repair
])
pw = np.array([
    'Apoptosis', 'P53_Pathway',
    'Apoptosis', 'P53_Pathway',
    'P53_Pathway', 'DNA_Repair',
    'DNA_Repair',
])

gs_ensg = GeneSets(genes=ensg_genes, pathways=pw)
print("Before mapping:")
print(f"  Genes: {gs_ensg.genes}")

# Convert Ensembl IDs to gene symbols
gs_ensg.map_genes(mapper, from_type='ensg', to_type='symbol')
print("\nAfter mapping:")
print(f"  Genes: {gs_ensg.genes}")

### 3c. Legacy: convert_from_to() (Requires Network)

`ExpressionProfile.convert_from_to()`, `GeneSets.convert_from_to()`, and `Heatmap.convert_from_to()` use Ensembl BioMart via `pybiomart` and require an internet connection:

```python
# Example (not run — requires network)
exp.convert_from_to('refseq', 'ensg', 'human')
gs.convert_from_to('ensg', 'gs', 'human')
```

**Recommendation:** Prefer `GeneMapper` for offline, reproducible workflows.

---
## 4. Bulk PAGE Analysis

The core `PAGE` class performs pathway enrichment analysis on bulk expression data.

### 4a. Setting Up PAGE

Load expression and gene set data, then initialize a `PAGE` object:

In [ ]:
# Load pre-binned expression data
expr_df = pd.read_csv(
    "../example_data/AP2S1.tab.gz",
    sep="\t", header=None, names=["gene", "bin"]
)
exp = ExpressionProfile(expr_df["gene"], expr_df["bin"], is_bin=True)

# Load GO Biological Process annotations (long-format: gene, pathway pairs)
ann = pd.read_csv(
    "../example_data/GO_BP_2021_index.txt.gz",
    sep="\t", header=None, names=["gene", "pathway"]
)
gs = GeneSets(ann["gene"], ann["pathway"])

print(exp)
print(gs)

Key `PAGE` parameters:

| Parameter | Default | Description |
|-----------|---------|-------------|
| `function` | `'cmi'` | `'cmi'` (conditional MI, corrects annotation bias) or `'mi'` (standard MI) |
| `n_shuffle` | `10000` | Number of permutations for significance testing |
| `alpha` | `0.005` | P-value threshold for informative pathways |
| `k` | `20` | Early-stopping: stop after k consecutive non-significant pathways |
| `filter_redundant` | `True` | Whether to remove redundant pathways (default: on) |
| `redundancy_ratio` | `5.0` | CMI/MI ratio threshold; pathways with all ratios above this are kept |
| `n_jobs` | `1` | Number of parallel threads |

### 4b. Running the Analysis

In [ ]:
# Use small n_shuffle for speed in this tutorial
p = PAGE(exp, gs, n_shuffle=50, k=7, filter_redundant=False)
results, heatmap = p.run()

print(f"Found {len(results)} significant pathways")

### 4c. Interpreting the Results DataFrame

The results DataFrame has these columns:
- **pathway** — pathway name
- **CMI** — conditional mutual information (or MI if `function='mi'`)
- **p-value** — empirical p-value from permutation test
- **Regulation pattern** — `1` for upregulated, `-1` for downregulated

In [ ]:
results.head(10)

### 4d. Visualizing the Heatmap

The `Heatmap` object shows per-bin enrichment patterns for significant pathways. Positive values (yellow/bright) indicate overrepresentation; negative (dark/purple) indicate underrepresentation.

In [ ]:
if heatmap is not None:
    heatmap.show(max_rows=20, title='GO BP Enrichment')

### 4e. Extracting Enriched Genes per Pathway

`get_enriched_genes()` returns a list of arrays, one per expression bin, showing which pathway genes fall in each bin:

In [ ]:
if len(results) > 0:
    pathway_name = results.iloc[0]['pathway']
    enriched = p.get_enriched_genes(pathway_name)
    print(f"Genes in '{pathway_name}' by expression bin:")
    for i, genes_in_bin in enumerate(enriched):
        if len(genes_in_bin) > 0:
            print(f"  Bin {i}: {genes_in_bin[:5]}{'...' if len(genes_in_bin) > 5 else ''}")

### 4f. Getting the Enrichment Score Matrix

`get_es_matrix()` returns a DataFrame with log10 hypergeometric p-values per pathway and bin:

In [ ]:
if heatmap is not None:
    es_matrix = p.get_es_matrix()
    print(f"ES matrix shape: {es_matrix.shape}")
    es_matrix.head()

### 4g. MI vs CMI Comparison

CMI (conditional mutual information) accounts for annotation bias by conditioning on gene membership counts. Let's compare MI and CMI results:

In [ ]:
# Run with MI
np.random.seed(42)
p_mi = PAGE(exp, gs, function='mi', n_shuffle=50, k=7)
results_mi, _ = p_mi.run()

# Run with CMI
np.random.seed(42)
p_cmi = PAGE(exp, gs, function='cmi', n_shuffle=50, k=7)
results_cmi, _ = p_cmi.run()

print(f"MI found {len(results_mi)} significant pathways")
print(f"CMI found {len(results_cmi)} significant pathways")

if len(results_mi) > 0 and len(results_cmi) > 0:
    mi_pathways = set(results_mi['pathway'])
    cmi_pathways = set(results_cmi['pathway'])
    print(f"\nOverlap: {len(mi_pathways & cmi_pathways)} pathways")
    print(f"MI-only: {len(mi_pathways - cmi_pathways)} pathways")
    print(f"CMI-only: {len(cmi_pathways - mi_pathways)} pathways")

### 4h. Redundancy Filtering

When `filter_redundant=True`, PAGE removes pathways that are redundant with already-accepted pathways, based on conditional mutual information between pathway memberships:

In [ ]:
np.random.seed(42)
p_red = PAGE(exp, gs, n_shuffle=50, k=7, filter_redundant=True)
results_red, _ = p_red.run()

print(f"Without redundancy filter: {len(results_cmi)} pathways")
print(f"With redundancy filter: {len(results_red)} pathways")

# Inspect which pathways were filtered and why
killed = p_red.get_redundancy_log()
if len(killed) > 0:
    print(f"\nFiltered {len(killed)} redundant pathways:")
    print(killed.head(10))

# full_results includes all informative pathways with a 'redundant' flag
print(f"\nAll informative pathways (including redundant):")
print(p_red.full_results.head(10))

### 4i. Reproducibility

For fully deterministic results, set a random seed and use a single thread:

In [ ]:
# Reload data fresh for a clean reproducibility test
expr_df_ = pd.read_csv("../example_data/AP2S1.tab.gz", sep="\t", header=None, names=["gene", "bin"])
ann_ = pd.read_csv("../example_data/GO_BP_2021_index.txt.gz", sep="\t", header=None, names=["gene", "pathway"])

np.random.seed(0)
exp1 = ExpressionProfile(expr_df_["gene"], expr_df_["bin"], is_bin=True)
gs1 = GeneSets(ann_["gene"], ann_["pathway"])
p1 = PAGE(exp1, gs1, n_shuffle=50, k=7, n_jobs=1)
r1, _ = p1.run()

np.random.seed(0)
exp2 = ExpressionProfile(expr_df_["gene"], expr_df_["bin"], is_bin=True)
gs2 = GeneSets(ann_["gene"], ann_["pathway"])
p2 = PAGE(exp2, gs2, n_shuffle=50, k=7, n_jobs=1)
r2, _ = p2.run()

if len(r1) > 0 and len(r2) > 0 and len(r1) == len(r2):
    match = (r1['pathway'].values == r2['pathway'].values).all()
    print(f"Results identical with same seed: {match}")
    print(f"Both runs found {len(r1)} significant pathways")
else:
    print(f"Run 1: {len(r1)} pathways, Run 2: {len(r2)} pathways")

In [ ]:
# Pick two known pathways from the gene sets
available_pathways = list(gs.pathways[:5])
print(f"Example pathways: {available_pathways}")

# Run manual analysis on specific pathways
p_manual = PAGE(exp, gs, n_shuffle=50, k=7)
manual_results, manual_hm = p_manual.run_manual(available_pathways[:2])
print(f"\nManual analysis results ({len(manual_results)} pathways):")
print(manual_results)

# Note: p-values are NaN since no permutation testing is performed
if manual_hm is not None:
    manual_hm.show(title='Manual Pathway Analysis')

### 4j. Manual Pathway Analysis

Use `run_manual()` to analyze specific pathways of interest without permutation testing or redundancy filtering. This is useful for inspecting known pathways:

---
## 5. Single-Cell PAGE Analysis

`SingleCellPAGE` computes per-cell pathway activity scores using MI/CMI, then tests spatial coherence of pathway scores across the cell manifold using Geary's C statistic.

### 5a. Creating Synthetic Single-Cell Data

We'll create a synthetic dataset with two cell clusters and pathways with known differential activity:

In [ ]:
np.random.seed(42)

n_cells = 200
n_genes_sc = 100
n_cluster1 = 100

gene_names_sc = np.array([f'Gene{i}' for i in range(n_genes_sc)])

# Base expression
X = np.random.rand(n_cells, n_genes_sc) * 5

# Cluster 1 (cells 0-99): upregulate genes 0-19 ("pathway A" genes)
X[:n_cluster1, :20] += 4

# Cluster 2 (cells 100-199): upregulate genes 20-39 ("pathway B" genes)
X[n_cluster1:, 20:40] += 4

# Create a 2D embedding (two clusters)
embedding = np.zeros((n_cells, 2))
embedding[:n_cluster1] = np.random.randn(n_cluster1, 2) + np.array([3, 0])
embedding[n_cluster1:] = np.random.randn(n_cells - n_cluster1, 2) + np.array([-3, 0])

# Cluster labels
labels = np.array(['Cluster_A'] * n_cluster1 + ['Cluster_B'] * (n_cells - n_cluster1))

print(f"Expression matrix: {X.shape}")
print(f"Embedding: {embedding.shape}")
print(f"Clusters: {np.unique(labels, return_counts=True)}")

In [ ]:
# Define gene sets with known cluster-specific activity
gs_genes = np.concatenate([
    gene_names_sc[:20],       # Pathway_A genes
    gene_names_sc[20:40],     # Pathway_B genes
    gene_names_sc[50:65],     # Pathway_C genes (no cluster bias)
])
gs_pathways = np.concatenate([
    np.repeat('Pathway_A', 20),
    np.repeat('Pathway_B', 20),
    np.repeat('Pathway_C', 15),
])

gs_sc = GeneSets(genes=gs_genes, pathways=gs_pathways)
print(gs_sc)

### 5b. Creating an AnnData Object

`SingleCellPAGE` works natively with AnnData objects (the standard for single-cell Python tools):

In [ ]:
import anndata

adata = anndata.AnnData(
    X=X,
    var=pd.DataFrame(index=gene_names_sc),
    obs=pd.DataFrame({'cluster': labels}),
)
adata.obsm['X_umap'] = embedding

print(adata)

### 5c. Running Per-Cell Scoring + Spatial Coherence

In [ ]:
np.random.seed(42)

sc_page = SingleCellPAGE(
    adata=adata,
    genesets=gs_sc,
    function='cmi',
    n_bins=5,
)
print(sc_page)

In [ ]:
# Run with reduced permutations for speed
sc_results = sc_page.run(n_permutations=200)
sc_results

### 5d. Interpreting Results

The results DataFrame contains:
- **pathway** — pathway name
- **consistency** — spatial autocorrelation score (C' = 1 - Geary's C). Higher values mean pathway scores are more spatially coherent across the cell manifold.
- **p-value** — empirical p-value from permutation test with size-matched random gene sets
- **FDR** — Benjamini-Hochberg corrected p-value

Pathways A and B should show high consistency (their activity is clustered), while Pathway C (random) should not:

In [ ]:
print(sc_results.to_string(index=False))

### 5e. Visualizations

In [ ]:
# Pathway scores on UMAP embedding
fig, axes = plt.subplots(1, 3, figsize=(16, 4))
for i, pw in enumerate(gs_sc.pathways):
    sc_page.plot_pathway_on_embedding(pw, embedding_key='X_umap', ax=axes[i])
plt.tight_layout()
plt.show()

In [ ]:
# Consistency ranking bar plot
sc_page.plot_consistency_ranking(top_n=10)
plt.tight_layout()
plt.show()

In [ ]:
# Heatmap of pathway scores across clusters
sc_page.plot_pathway_heatmap(adata.obs['cluster'])
plt.tight_layout()
plt.show()

### 5f. Accessing the Per-Cell Score Matrix

The per-cell pathway scores are stored as a `(n_cells, n_pathways)` matrix:

In [ ]:
print(f"Score matrix shape: {sc_page.scores.shape}")

# Convert to a DataFrame for easy inspection
scores_df = pd.DataFrame(
    sc_page.scores,
    columns=sc_page.pathway_names,
)
scores_df.describe()

### 5g. Neighborhood Mode

`run_neighborhoods()` aggregates cells by cluster label (or random micro-pools), computes mean expression per group, and runs standard bulk PAGE on each group:

In [ ]:
np.random.seed(42)
summary, group_results = sc_page.run_neighborhoods(labels=adata.obs['cluster'])
print("Summary:")
print(summary.to_string(index=False))

for group, (res_df, hm) in group_results.items():
    print(f"\nGroup '{group}': {len(res_df)} significant pathways")
    if len(res_df) > 0:
        print(res_df.head())

### 5h. Alternative Inputs

You don't need an AnnData object — `SingleCellPAGE` also accepts raw numpy arrays or a precomputed KNN connectivity matrix:

In [ ]:
# From numpy arrays
sc_numpy = SingleCellPAGE(
    expression=X,
    genes=gene_names_sc,
    genesets=gs_sc,
    function='mi',
    n_bins=5,
)
print(sc_numpy)

In [ ]:
# With precomputed connectivity (e.g., from scanpy)
from pypage.spatial import build_knn_graph

W = build_knn_graph(X, k=15)
sc_precomputed = SingleCellPAGE(
    expression=X,
    genes=gene_names_sc,
    genesets=gs_sc,
    connectivity=W,
)
print(sc_precomputed)

---
## 6. Complete Workflow: GMT + GeneMapper + PAGE

This section demonstrates a realistic end-to-end workflow that combines GMT loading, gene ID mapping, and PAGE analysis.

In [ ]:
# Step 1: Load gene sets from GMT
gs_workflow = GeneSets.from_gmt("../example_data/example.gmt")
print(f"Loaded {gs_workflow.n_pathways} pathways with {gs_workflow.n_genes} genes")
print(f"Gene ID format: {gs_workflow.genes[:5]}")

In [ ]:
# Step 2: Create expression data using gene symbols (matching the GMT)
# In a real workflow, you might need to convert IDs first.
# Here the GMT already uses symbols, so we create matching expression data.

all_genes = gs_workflow.genes.copy()
# Add some extra genes not in the gene sets (realistic scenario)
extra_genes = np.array([f'ExtraGene{i}' for i in range(50)])
all_expr_genes = np.concatenate([all_genes, extra_genes])
expression_values = np.random.randn(len(all_expr_genes))

exp_workflow = ExpressionProfile(all_expr_genes, expression_values, n_bins=5)
print(f"Expression: {exp_workflow.n_genes} genes")

In [ ]:
# Step 3: Run PAGE
np.random.seed(42)
p_workflow = PAGE(exp_workflow, gs_workflow, n_shuffle=50, k=5)
results_workflow, hm_workflow = p_workflow.run()

print(f"Shared genes: {len(p_workflow.shared_genes)}")
print(f"Significant pathways: {len(results_workflow)}")
if len(results_workflow) > 0:
    print(results_workflow)

In [ ]:
# Step 4: Visualize
if hm_workflow is not None:
    hm_workflow.show(title='GMT Workflow Results')

---
## 7. Parameter Reference

### PAGE Parameters

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| `expression` | `ExpressionProfile` | — | Expression data (required) |
| `genesets` | `GeneSets` | — | Gene set annotations (required) |
| `function` | `str` | `'cmi'` | `'cmi'` for conditional MI (recommended), `'mi'` for standard MI |
| `n_shuffle` | `int` | `10000` | Number of permutations for significance testing |
| `alpha` | `float` | `0.005` | P-value significance threshold |
| `k` | `int` | `20` | Early stopping: consecutive non-significant pathways before stopping |
| `filter_redundant` | `bool` | `True` | Remove redundant pathways using CMI |
| `redundancy_ratio` | `float` | `5.0` | CMI/MI ratio threshold; pathways with all ratios above this are kept |
| `n_jobs` | `int` | `1` | Number of parallel threads |

### SingleCellPAGE Parameters

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| `adata` | `AnnData` | `None` | AnnData object (alternative: `expression` + `genes`) |
| `expression` | `np.ndarray` | `None` | (n_cells, n_genes) expression matrix |
| `genes` | `np.ndarray` | `None` | Gene names array |
| `genesets` | `GeneSets` | — | Gene set annotations (required) |
| `function` | `str` | `'cmi'` | `'cmi'` or `'mi'` |
| `n_bins` | `int` | `10` | Number of bins for expression discretization |
| `n_neighbors` | `int` | `ceil(sqrt(n_cells))` | Number of KNN neighbors (capped at 100) |
| `connectivity` | `sparse matrix` | `None` | Precomputed cell-cell connectivity |

### GeneMapper Parameters

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| `species` | `str` | `'human'` | `'human'` or `'mouse'` |
| `cache_dir` | `str` | `'~/.pypage/'` | Directory for the cached mapping file |

### GeneSets.from_gmt() Parameters

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| `gmt_file` | `str` | — | Path to `.gmt` or `.gmt.gz` file |
| `n_bins` | `int` | `3` | Number of bins for membership binning |
| `min_size` | `int` | `None` | Minimum pathway size (filter after loading) |
| `max_size` | `int` | `None` | Maximum pathway size (filter after loading) |

In [ ]:
# Load GMT with gene symbols
gs_convert = GeneSets.from_gmt("../example_data/example.gmt")
print(f"GMT genes (symbols): {gs_convert.genes[:5]}")

# Suppose our expression data uses Ensembl IDs.
# We can convert the GeneSets to use Ensembl IDs to match,
# or convert the expression data. Let's convert GeneSets -> Ensembl:

# First, let's see which GMT genes are in our mock mapper
test_result, test_unmapped = mapper.convert(
    gs_convert.genes, from_type='symbol', to_type='ensg'
)
n_mapped = sum(1 for x in test_result if x is not None)
print(f"\nMappable genes: {n_mapped}/{len(gs_convert.genes)}")

# Convert gene symbols -> Ensembl IDs
gs_convert.map_genes(mapper, from_type='symbol', to_type='ensg')
print(f"After conversion: {gs_convert.n_genes} genes (Ensembl IDs)")
print(f"Example genes: {gs_convert.genes[:5]}")

---
## 7. Parameter Reference

### PAGE Parameters

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| `expression` | `ExpressionProfile` | — | Expression data (required) |
| `genesets` | `GeneSets` | — | Gene set annotations (required) |
| `function` | `str` | `'cmi'` | `'cmi'` for conditional MI (recommended), `'mi'` for standard MI |
| `n_shuffle` | `int` | `1000` | Number of permutations for significance testing |
| `alpha` | `float` | `0.01` | P-value significance threshold |
| `k` | `int` | `10` | Early stopping: consecutive non-significant pathways before stopping |
| `filter_redundant` | `bool` | `False` | Remove redundant pathways using CMI |
| `redundancy_ratio` | `float` | `0.1` | Redundancy threshold (higher = stricter filtering) |
| `n_jobs` | `int` | `1` | Number of parallel threads |

### SingleCellPAGE Parameters

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| `adata` | `AnnData` | `None` | AnnData object (alternative: `expression` + `genes`) |
| `expression` | `np.ndarray` | `None` | (n_cells, n_genes) expression matrix |
| `genes` | `np.ndarray` | `None` | Gene names array |
| `genesets` | `GeneSets` | — | Gene set annotations (required) |
| `function` | `str` | `'cmi'` | `'cmi'` or `'mi'` |
| `n_bins` | `int` | `10` | Number of bins for expression discretization |
| `n_neighbors` | `int` | `ceil(sqrt(n_cells))` | Number of KNN neighbors (capped at 100) |
| `connectivity` | `sparse matrix` | `None` | Precomputed cell-cell connectivity |

### GeneMapper Parameters

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| `species` | `str` | `'human'` | `'human'` or `'mouse'` |
| `cache_dir` | `str` | `'~/.pypage/'` | Directory for the cached mapping file |

### GeneSets.from_gmt() Parameters

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| `gmt_file` | `str` | — | Path to `.gmt` or `.gmt.gz` file |
| `n_bins` | `int` | `3` | Number of bins for membership binning |
| `min_size` | `int` | `None` | Minimum pathway size (filter after loading) |
| `max_size` | `int` | `None` | Maximum pathway size (filter after loading) |

---
## 8. Next Steps

- **API Reference:** See `MANUAL.md` for the complete API documentation.
- **More examples:** See `notebooks/pyPAGE_bulk_and_sc_example.ipynb` and `notebooks/single_cell_page_tutorial.ipynb` for additional worked examples.
- **Citation:** If you use pyPAGE in your research, please cite:

> Bakulin A, Teyssier NB, Kampmann M, Khoroshkin M, Goodarzi H (2024)  
> *pyPAGE: A framework for Addressing biases in gene-set enrichment analysis — A case study on Alzheimer's disease.*  
> PLoS Computational Biology 20(9): e1012346.  
> https://doi.org/10.1371/journal.pcbi.1012346

In [ ]:
# Cleanup mock cache
import shutil
shutil.rmtree(mock_cache_dir, ignore_errors=True)
print("Tutorial complete!")